In [227]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np

import libry as ry
import time
print(cv.__version__)

4.5.3


In [228]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/ex05_redball.g")
D = RealWorld.view()

ball = RealWorld.getFrame("ball")
ball.setContact(1)

In [229]:
#RealWorld.addFrame(camera_fixpoint)

In [230]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.bullet, verbose=2)
S.addSensor("camera")

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:68(0) starting bullet engine ...

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:81(0) ... done starting bullet engine
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:68(0) starting bullet engine ...

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:85(0) creating Configuration within bullet ...
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:81(0) ... done starting bullet engine

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:addLink:259(0) adding link anchored at 'world' as static
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:85(0) creating Configuration within bullet ...

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:addLink:259(0) addin

/home/tim/git/robotics-course_tim/rai/rai/Kin/kin_bullet.cpp:BulletInterface:92(0) ... done creating Configuration within bullet


In [231]:
# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
#S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [232]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
cameraFrame = C.frame("camera")
D2 = C.view()

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [233]:

config_obj_name = "conf_ball"

# delete if necceary
#C.delFrame(config_obj_name)

obj = C.addFrame(config_obj_name,"camera")

obj.setShape(ry.ST.sphere, [.03])
obj.setPosition([0., .1, 1.6])
obj.setColor([1.,0,0])


objectPosition = RealWorld.getFrame("ball").getPosition()

#set the model object to percept
obj.setPosition(objectPosition)



In [234]:
def _segment_redball(rgb):
    rgb = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
    hsv = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)

    mask1 = cv.inRange(hsv, (0, 120, 70), (10, 255, 255))
    mask2 = cv.inRange(hsv, (170, 120, 70), (180, 255, 255))

    # if len(mask1)>0: cv.imshow('OPENCV - mask1',  mask1)
    # if len(mask2)>0: cv.imshow('OPENCV - mask2',  mask2)
    mask = mask1 + mask2

    # create empty mask
    filtered_mask = np.zeros(mask.shape, np.uint8)

    # if len(filtered_mask)>0: cv.imshow('OPENCV - filtered_mask',  filtered_mask)
    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        # find largest contour
        largest, idx = 0, None
        for i, c in enumerate(contours):
            # remove noise
            if c.shape[0] < 10: continue
            if cv.contourArea(c) > largest:
                largest = cv.contourArea(c)
                idx = i

        cv.drawContours(filtered_mask, contours[idx], -1, (255, 255, 255), -1)
    return filtered_mask


def _image_pointcloud(depth, mask):
    # if len(mask) > 0: cv.imshow('OPENCV - mask1', mask * 255)
    # cv.waitKey(100)
    mask_pixels = np.where(mask > 0)
    pointcloud = np.empty((mask_pixels[0].shape[0], 3))
    pointcloud[:, 0] = mask_pixels[1]  # x pixels
    pointcloud[:, 1] = mask_pixels[0]  # y pixels
    pointcloud[:, 2] = depth[mask_pixels[0], mask_pixels[1]]
    return pointcloud


def _meter_pointcloud(pixel_points, fxfypxpy):
    points = np.empty(np.shape(pixel_points))
    for i, p in enumerate(pixel_points):
        x = p[0]
        y = p[1]
        d = p[2]

        px = fxfypxpy[-2]
        py = fxfypxpy[-1]

        x_ = d * (x - px) / fxfypxpy[0]
        y_ = -d * (y - py) / fxfypxpy[1]
        z_ = -d
        points[i] = [x_, y_, z_]
    return points


def find_ball(rgb, depth, fxfypxpy):
    mask = _segment_redball(rgb)
    pixel_points = _image_pointcloud(depth, mask)
    obj_points = _meter_pointcloud(pixel_points, fxfypxpy)
    return obj_points, mask

In [235]:
points = []
tau = .01

background = None

gripping = False
grasped = False

objectPosition = [0., .1, 1.6]


for t in range(200):
  time.sleep(0.01)
  #grab sensor readings from the simulation
  q = S.get_q()
  if t%10 == 0:
      [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
      points = S.depthData2pointCloud(depth, fxfypxpy)
      cameraFrame.setPointCloud(points, rgb)
      pos_bkp = objectPosition

      # get points of ball with opencv
      obj_points, mask = find_ball(rgb,depth,fxfypxpy)

      # get position of ball with mean of points
      objectPosition = obj_points.mean(axis=0)

      if np.isnan(objectPosition).any():
        objectPosition = pos_bkp

      #print(objectPosition)

      #if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
      #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

  
  #TOTAL CHEAT: grab the true position from the RealWorld
  #objectPosition = RealWorld.getFrame("obj0").getPosition()

  #set the model object to percept (relative to camera because the position from the pointcloud is in camera coord.)
  obj.setRelativePosition(objectPosition)

  C.setJointState(q) #set your robot model to match the real q


  if t>=100 and not gripping:
    
    # we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
    IK = C.komo_IK(False)
    IK.add_qControlObjective([],1,0.5)
    IK.addObjective([1.], 
                    ry.FS.positionDiff, 
                    ["R_gripper", config_obj_name], 
                    ry.OT.eq, 
                     [1e2],
                    [0.,0.,0]);
    IK.addObjective([1.], 
                    ry.FS.distance, 
                    ["R_gripper", config_obj_name], 
                    ry.OT.sos, 
                     [1e2],
                    [-.04]);
    IK.addObjective([1.], 
                    ry.FS.vectorY, 
                    ["R_gripper"], 
                    ry.OT.sos, 
                    [1e2],
                    [0.,0.,-1]);

    #IK.addOb

    # reoptimize
    IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
    #print(IK.getReport()

    C.setFrameState(IK.getFrameState(0))


    # get joint states
    q = C.getJointState()

  [y,J] = C.evalFeature(ry.FS.positionDiff, ["R_gripper", config_obj_name])

  if(not gripping and (np.linalg.norm(y) < .02)):
    S.closeGripper("R_gripper",width=0.025, speed = 3.)
    gripping = True
  

  if S.getGripperIsGrasping("R_gripper"): print("Trueeeeeee")

  if(gripping and S.getGripperIsGrasping("R_gripper")):
    print("GRASPED!******************************************************++++")
    break;
  
  #send no controls to the simulation
  S.step(q, tau, ry.ControlMode.position)




/tmp/ipykernel_12509/773753771.py:26: RuntimeWarning: Mean of empty slice.
  objectPosition = obj_points.mean(axis=0)


** KOMO::run solver:dense collisions:0 x-dim:14 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:14 #frames:196
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:closeGripper:312(1) closing gripper without near object (if this is not what you expect, did you setContact(1) for the object you want to grasp?)

** optimization time:0.001778 (kin:8.7e-05 coll:0 feat:0.000771 newton: 0.000178) setJointStateCount:20
   sos:50.0152 ineq:0 eq:0.0519623
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:closeGripper:312(1) closing gripper without near object (if this is not what you expect, did you setContact(1) for the object you want to grasp?)
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:modConfiguration:568(1) terminating closing gripper (limit) - nothing grasped

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:modConfiguration:568(1) terminating closing gripper (limit) - nothing g

In [236]:
# calculate a lift of the ball

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 3., True)
komo.addObjective([1.], ry.FS.position, ["R_gripper"], ry.OT.sos, [1e2],[-.0,.1,1.6])
komo.addObjective([1.], ry.FS.qItself, [], ry.OT.eq, [1e2], order=1)
komo.addObjective([1.], 
                    ry.FS.vectorY, 
                    ["R_gripper"], 
                    ry.OT.sos, 
                    [1e2],
                    [0.,0.,-1]);
komo.optimize()
#komo.getReport()
#V = komo.view_play(False, 1)


** KOMO::run solver:sparse collisions:1 x-dim:280 T:20 k:2 phases:1 stepsPerPhase:20 tau:0.15  #timeSlices:22 #totalDOFs:280 #frames:2156
** optimization time:0.091234 (kin:0.002816 coll:0.070253 feat:0.00888 newton: 0.003761) setJointStateCount:15
   sos:1.76601 ineq:0 eq:1.29633e-05


In [237]:

# here loop for upwards move
#input()
for frame,tau in zip(komo.getPathFrames(),komo.getPathTau()):
    time.sleep(tau)



    [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    
    C.setFrameState(frame)

    q= C.getJointState()
    
    #send position to the simulation
    S.step(q, tau, ry.ControlMode.position)

tau = 0.01
for t in range(100):
    time.sleep(tau)

    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
    
        #if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        #if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)
    
    if S.getGripperWidth("R_gripper") < 0.02:
        S.openGripper("R_gripper",speed=3.)

    S.step([], tau, ry.ControlMode.none)

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper
** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper
**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object

**ry-c++-log** /home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:264(1) initiating opening gripper R_gripper** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.cpp:openGripper:248(1) gripper 'R_gripper' does not hold an object


** INFO:/home/tim/git/robotics-course_tim/rai/rai/Kin/simulation.

In [238]:
cv.destroyAllWindows()